In [1]:

!pip install -qU langchain-chroma
!pip install -qU langchain-google-genai
!pip install -qU langchain-community
!pip install python-dotenv
!pip install sentence-transformers

In [1]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
from dotenv import load_dotenv
import pandas as pd


In [2]:

df= pd.read_csv("/Users/jigyanshupati/semantic_researc_paper/arXiv_scientific_dataset_cleaned.csv")
df["tagged_summary"].to_csv("tagged_summary.txt", sep="\n", index=False, header=False)


In [4]:
import re
from langchain.schema import Document

# Read the file
with open("tagged_summary.txt", "r", encoding="utf-8") as f:
    text = f.read()

# Match entries inside quotes
entries = re.findall(r'"(.*?)"', text, re.DOTALL)

# Keep only first 100 entries
entries = entries[:100]

# Create documents
documents = [Document(page_content=entry.strip()) for entry in entries]

print("Total chunks:", len(documents))


Total chunks: 100


In [5]:
from sentence_transformers import SentenceTransformer

# Step 1: Extract text
texts = [doc.page_content for doc in documents]

# Step 2: Load the model
model_name = 'all-MiniLM-L6-v2'
model = SentenceTransformer(model_name)

# Step 3: Encode sequentially
embeddings = model.encode(texts, show_progress_bar=True)

print(f"Total embeddings generated: {len(embeddings)}")

/Users/jigyanshupati/semantic_researc_paper/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Batches: 100%|██████████| 4/4 [00:02<00:00,  1.65it/s]

Total embeddings generated: 100


In [13]:
import uuid
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
from langchain.schema import Document
import re

# -------- Step 0: Read and prepare documents --------
with open("tagged_summary.txt", "r", encoding="utf-8") as f:
    text = f.read()

# Extract text entries inside quotes
entries = re.findall(r'"(.*?)"', text, re.DOTALL)

# Limit to first 100 entries (optional)
entries = entries[:500]

# Create Document objects
documents = [Document(page_content=entry.strip()) for entry in entries]

# Extract plain texts from documents
texts = [doc.page_content for doc in documents]

# -------- Step 1: Load sentence transformer model and encode --------
model_name = 'all-MiniLM-L6-v2'
model = SentenceTransformer(model_name)
embeddings = model.encode(texts, show_progress_bar=True)

print(f"Total documents: {len(documents)}")
print(f"Total embeddings generated: {len(embeddings)}")

# -------- Step 2: Initialize ChromaDB Client safely --------
try:
    client
except NameError:
    client = chromadb.Client(Settings(anonymized_telemetry=False))

# -------- Step 3: Get or create collection --------
collection = client.get_or_create_collection(name="tagged_summary_collection")

# -------- Step 4: Prepare ids and metadata --------
ids = [str(uuid.uuid4()) for _ in texts]
metadatas = [{"source": "tagged_summary"} for _ in texts]

# -------- Step 5: Add documents and embeddings to collection --------
collection.add(
    documents=texts,
    embeddings=embeddings.tolist() if hasattr(embeddings, 'tolist') else embeddings,
    metadatas=metadatas,
    ids=ids
)

print("Documents and embeddings added to Chroma collection successfully.")

Batches: 100%|██████████| 16/16 [00:07<00:00,  2.07it/s]


Total documents: 500
Total embeddings generated: 500
Documents and embeddings added to Chroma collection successfully.


In [22]:
query = "dependency-directed backtracking in search"
# Step 6: Encode the query with the same model
query_embedding = model.encode([query])

# Step 7: Query the collection with the embedding vector
results = collection.query(
    query_embeddings=query_embedding.tolist(),  # must be a list of lists
    n_results=10
)

# The results dict contains 'ids', 'documents', 'metadatas', 'distances'
results

{'ids': [['4c231385-bdaf-4635-a924-b2843d1a90b5',
   'aeb234ad-9ed3-4eb4-b85c-a697cdb0ff4a',
   '853c819b-2353-4a7d-b2ea-564df729fcd7',
   '756c4fc0-92e0-4331-8c1c-4bbeb68c7558',
   '8aa21f09-4857-4df5-995c-e9346df77ee6',
   '540fafc1-baca-49f1-85e7-a9a1954cab81',
   'eefa72c9-27ee-444f-8a6b-a2cca6052106',
   '68b8e361-16ae-4ffd-8da5-792a37d7b0ab',
   '984a5f04-062b-4d03-b118-c6eeec914d95',
   'ed219d6f-84d6-4d03-b64e-9824251ae22c']],
 'embeddings': None,
 'documents': [['cs-9308101v1 Because of their occasional need to return to shallow points in a search\ntree, existing backtracking methods can sometimes erase meaningful progress\ntoward solving a search problem. In this paper, we present a method by which\nbacktrack points can be moved deeper in the search space, thereby avoiding this\ndifficulty. The technique developed is a variant of dependency-directed\nbacktracking that uses only polynomial space while still providing useful\ncontrol information and retaining the completeness g

In [23]:
import pandas as pd
df= pd.read_csv("/Users/jigyanshupati/semantic_researc_paper/arXiv_scientific dataset.csv")
first_doc = results['documents'][0][0]
doc_id = first_doc.split()[0].strip()
df[df["id"] == doc_id]



,id,title,category,category_code,published_date,updated_date,authors,first_author,summary,summary_word_count
0,cs-9308101v1,Dynamic Backtracking,Artificial Intelligence,cs.AI,8/1/93,8/1/93,['M. L. Ginsberg'],'M. L. Ginsberg',Because of their occasional need to return to ...,79


In [ ]:
import gradio as gr
import pandas as pd
import uuid
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer

# ---- Step 1: Load SentenceTransformer model ----
model = SentenceTransformer("all-MiniLM-L6-v2")

# ---- Step 2: Initialize ChromaDB client and collection ----
client = chromadb.Client(Settings(anonymized_telemetry=False))
collection = client.get_or_create_collection(name="tagged_summary_collection")

# ---- Step 3: Define CSV path ----
csv_path = "/Users/jigyanshupati/semantic_researc_paper/arXiv_scientific dataset.csv"

# ---- Step 4: Define search function ----
def get_recommendations(query: str, k: int = 5) -> pd.DataFrame:
    """
    Perform semantic search and return top-k results from the CSV.
    """
    # Encode the query
    query_embedding = model.encode([query])

    # Query ChromaDB
    results = collection.query(
        query_embeddings=query_embedding.tolist(),
        n_results=k
    )

    # Extract document IDs from the result
    doc_ids = [doc.split()[0].strip() for doc in results['documents'][0]]

    # Load the CSV dataset
    df = pd.read_csv(csv_path)

    # Filter by document ID
    matched_df = df[df["id"].isin(doc_ids)]

    return matched_df

# ---- Step 5: Gradio UI ----
def search_interface(query):
    return get_recommendations(query, k=5)

gr.Interface(
    fn=search_interface,
    inputs=gr.Textbox(label="Search Query"),
    outputs=gr.Dataframe(label="Top Matches"),
    title="Semantic Paper Recommender",
    description="Enter a query to retrieve the most relevant research papers."
).launch()

In [31]:
get_recommendations(query = "dependency-directed backtracking in search", k = 10)

,id,title,category,category_code,published_date,updated_date,authors,first_author,summary,summary_word_count
0,cs-9308101v1,Dynamic Backtracking,Artificial Intelligence,cs.AI,8/1/93,8/1/93,['M. L. Ginsberg'],'M. L. Ginsberg',Because of their occasional need to return to ...,79
39,cs-9512101v1,OPUS: An Efficient Admissible Algorithm for Un...,Artificial Intelligence,cs.AI,12/1/95,12/1/95,['G. I. Webb'],'G. I. Webb',OPUS is a branch and bound search algorithm th...,91
49,cs-9603101v1,Quantum Computing and Phase Transitions in Com...,Artificial Intelligence,cs.AI,3/1/96,3/1/96,['T. Hogg'],'T. Hogg',We introduce an algorithm for combinatorial se...,111
91,cs-9712102v1,Bidirectional Heuristic Search Reconsidered,Artificial Intelligence,cs.AI,12/1/97,12/1/97,"['H. Kaindl', 'G. Kainz']",'H. Kaindl',The assessment of bidirectional heuristic sear...,204
207,cs-0305001v1,A Framework for Searching AND/OR Graphs with C...,Artificial Intelligence,cs.AI,5/1/03,5/1/03,"['Ambuj Mahanti', 'Supriyo Ghose', 'Samir K. S...",'Ambuj Mahanti',Search in cyclic AND/OR graphs was traditional...,158
276,cs-0407040v1,Decomposition Based Search - A theoretical and...,Artificial Intelligence,cs.AI,7/16/04,7/16/04,"['W. J. van Hoeve', 'M. Milano']",'W. J. van Hoeve',In this paper we present and evaluate a search...,152
277,cs-0407042v1,Postponing Branching Decisions,Artificial Intelligence,cs.AI,7/16/04,7/16/04,"['Willem Jan van Hoeve', 'Michela Milano']",'Willem Jan van Hoeve',Solution techniques for Constraint Satisfactio...,127
